In [1]:
import os
import json
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
pip install dgl dglgo -f https://data.dgl.ai/wheels/repo.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.dgl.ai/wheels/repo.html
     |████████████████████████████████| 6.2 MB 6.5 MB/s 
     |████████████████████████████████| 63 kB 2.4 MB/s 
     |████████████████████████████████| 281 kB 74.9 MB/s 
     |████████████████████████████████| 78 kB 8.3 MB/s 
     |████████████████████████████████| 103 kB 65.7 MB/s 
     |████████████████████████████████| 596 kB 71.5 MB/s 
     |████████████████████████████████| 109 kB 73.6 MB/s 
     |████████████████████████████████| 51 kB 636 kB/s 
     |████████████████████████████████| 45 kB 4.0 MB/s 
     |████████████████████████████████| 22.7 MB 6.1 MB/s 
     |████████████████████████████████| 42 kB 1.1 MB/s 
     |████████████████████████████████| 3.2 MB 52.8 MB/s 
     |████████████████████████████████| 546 kB 68.9 MB/s 
     |████████████████████████████████| 84 kB 4.5 MB/s 
     |████████████████████████████████| 100 kB 

In [ ]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from dgl import DGLGraph
import re
import scipy.sparse as sp

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [ ]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

In [ ]:
ratings = []
with open(os.path.join('/content/gdrive/MyDrive/Colab Notebooks/인턴/ml-1m/', 'ratings.dat'), encoding='latin1') as f:
    for l in f:
        user_id, movie_id, rating, timestamp = [int(_) for _ in l.split('::')]
        ratings.append({
            'user_id': user_id,
            'movie_id': movie_id,
            'rating': rating,
            'timestamp': timestamp,
        })
ratings = pd.DataFrame(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [ ]:
len(ratings.index)

1000209

In [ ]:
nodeidx = {}
ratings['user_id'].unique()

array([   1,    2,    3, ..., 6038, 6039, 6040])

In [ ]:
min(list(ratings['movie_id'].unique()))

1

In [ ]:
max(list(ratings['movie_id'].unique()))

3952

In [ ]:
np_movieid = ratings['movie_id'].values
np_movieid

array([1193,  661,  914, ...,  562, 1096, 1097])

In [ ]:
npplus = np.full((1000209,),6040)

In [ ]:
#min(list(np_movieid + npplus )),max(list(np_movieid + npplus ))

In [ ]:
ratings['movie_id_plus'] = list(np_movieid +  npplus)
ratings.head()

,user_id,movie_id,rating,timestamp,movie_id_plus
0,1,1193,5,978300760,7233
1,1,661,3,978302109,6701
2,1,914,3,978301968,6954
3,1,3408,4,978300275,9448
4,1,2355,5,978824291,8395


In [ ]:
'''
인덱스를 따로 부여할때 
idx = 0 
uniuser = ratings['user_id'].unique()
unimovie = ratings['movie_id_plus'].unique()

for i in range(len(uniuser)):
  nodeidx[idx] = uniuser[i]
  idx += 1

for i in range(len(unimovie)):
  nodeidx[idx] = unimovie[i]
  idx += 1

nodeidx'''

"\n인덱스를 따로 부여할때 \nidx = 0 \nuniuser = ratings['user_id'].unique()\nunimovie = ratings['movie_id_plus'].unique()\n\nfor i in range(len(uniuser)):\n  nodeidx[idx] = uniuser[i]\n  idx += 1\n\nfor i in range(len(unimovie)):\n  nodeidx[idx] = unimovie[i]\n  idx += 1\n\nnodeidx"

In [ ]:
nodeidx = {}
for i in range(9993):
  nodeidx[i] = i

In [ ]:
ratings.head(10)

,user_id,movie_id,rating,timestamp,movie_id_plus
0,1,1193,5,978300760,7233
1,1,661,3,978302109,6701
2,1,914,3,978301968,6954
3,1,3408,4,978300275,9448
4,1,2355,5,978824291,8395
5,1,1197,3,978302268,7237
6,1,1287,5,978302039,7327
7,1,2804,5,978300719,8844
8,1,594,4,978302268,6634
9,1,919,4,978301368,6959


In [ ]:
#!jupyter notebook --NotebookApp.iopub_data_rate_limit=100000000

In [ ]:
users = []
movies = []

for i in range(len(ratings.index)):
  user = ratings.iloc[i,0]
  users.append(nodeidx[user])

  movie = ratings.iloc[i, 4]
  movies.append(nodeidx[movie])


In [ ]:
edges_src = torch.from_numpy(np.array(users))
edges_dst = torch.from_numpy(np.array(movies))

In [ ]:
graph = dgl.graph((edges_src, edges_dst))
print(graph)

Graph(num_nodes=9993, num_edges=1000209,
      ndata_schemes={}
      edata_schemes={})


In [ ]:
edge_features = torch.from_numpy(ratings['rating'].to_numpy())
len(edge_features)

1000209

In [ ]:
graph.edata['w'] = edge_features

In [ ]:
print(graph)

Graph(num_nodes=9993, num_edges=1000209,
      ndata_schemes={}
      edata_schemes={'w': Scheme(shape=(), dtype=torch.int64)})


In [ ]:
graph.nodes()

tensor([   0,    1,    2,  ..., 9990, 9991, 9992])

https://docs.dgl.ai/tutorials/blitz/4_link_predict.html?highlight=negative

In [ ]:
#device = 'cpu'  

In [ ]:
def initializer(emb):
  torch.nn.init.xavier_uniform_(emb)
  return emb

In [ ]:
emb_size = 9993

In [ ]:
emb = dgl.nn.NodeEmbedding(graph.number_of_nodes(), emb_size, 'emb', init_func=initializer)
optimizer = dgl.optim.SparseAdam([emb], lr=0.001)

In [ ]:
#device = torch.device("cuda")
device = torch.device('cpu')
#feat = emb(graph.nodes(), device)
#feat 

In [ ]:
u, v = graph.edges()
eids = np.arange(graph.number_of_edges()) #엣지를 가지고 train, test로 나누어줌

eids = np.random.permutation(eids)
test_size = int(len(eids) * 0.1)    #엣지 중 10%를 택함
train_size = graph.number_of_edges() - test_size

test_pos_u, test_pos_v = u[eids[:test_size]], v[eids[:test_size]]
train_pos_u, train_pos_v = u[eids[test_size:]], v[eids[test_size:]]

adj = sp.coo_matrix((np.ones(len(u)), (u.numpy(), v.numpy())), shape=(9993,9993)      )
adj_neg = 1 - adj.todense() - np.eye(graph.number_of_nodes())
neg_u, neg_v = np.where(adj_neg != 0)

neg_eids = np.random.choice(len(neg_u), graph.number_of_edges())
test_neg_u, test_neg_v = neg_u[neg_eids[:test_size]], neg_v[neg_eids[:test_size]]
train_neg_u, train_neg_v = neg_u[neg_eids[test_size:]], neg_v[neg_eids[test_size:]]

train_g = dgl.remove_edges(graph, eids[:test_size])

train_pos_g = dgl.graph((train_pos_u, train_pos_v), num_nodes=graph.number_of_nodes())
train_neg_g = dgl.graph((train_neg_u, train_neg_v), num_nodes=graph.number_of_nodes())
test_pos_g = dgl.graph((test_pos_u, test_pos_v), num_nodes=graph.number_of_nodes())
test_neg_g = dgl.graph((test_neg_u, test_neg_v), num_nodes=graph.number_of_nodes())

In [ ]:
from dgl.nn import SAGEConv

# ----------- 2. create model -------------- #
# build a two-layer GraphSAGE model
class GraphSAGE(nn.Module):
    def __init__(self, in_feats, h_feats):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats, 'mean')
        self.conv2 = SAGEConv(h_feats, h_feats, 'mean')
    
    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

In [ ]:
import dgl.function as fn

class DotPredictor(nn.Module):
    def forward(self, g, h):
        with g.local_scope():
            g.ndata['h'] = h
            # Compute a new edge feature named 'score' by a dot-product between the
            # source node feature 'h' and destination node feature 'h'.
            g.apply_edges(fn.u_dot_v('h', 'h', 'score'))
            # u_dot_v returns a 1-element vector for each edge so you need to squeeze it.
            return g.edata['score'][:, 0]

In [ ]:
model = GraphSAGE(emb_size, 16)

# You can replace DotPredictor with MLPPredictor.
#pred = MLPPredictor(16)
pred = DotPredictor()

def compute_loss(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score])
    labels = torch.cat([torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])])
    return F.binary_cross_entropy_with_logits(scores, labels)

def compute_auc(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score]).numpy()
    labels = torch.cat(
        [torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]).numpy()
    return roc_auc_score(labels, scores)

In [ ]:
import itertools
optimizer = torch.optim.Adam(itertools.chain(model.parameters(), pred.parameters()), lr=0.01)

In [ ]:
feat2 = torch.zeros(9993, 9993) + torch.eye(9993)
feat2

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.]])

In [ ]:
all_logits = []
feat = emb(graph.nodes(), device)

for e in range(100):
    # forward
    #feat = emb(graph.nodes(), device)
    h = model(train_g, feat)
    pos_score = pred(train_pos_g, h)
    neg_score = pred(train_neg_g, h)
    loss = compute_loss(pos_score, neg_score)
    
    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if e % 5 == 0:
        print('In epoch {}, loss: {}'.format(e, loss))

In epoch 0, loss: 0.6932975053787231
In epoch 5, loss: 0.6367460489273071
In epoch 10, loss: 0.5887269973754883
In epoch 15, loss: 0.5708048343658447
In epoch 20, loss: 0.5620604157447815
In epoch 25, loss: 0.5567994713783264
In epoch 30, loss: 0.5516948699951172
In epoch 35, loss: 0.5463836789131165
In epoch 40, loss: 0.5414385199546814
In epoch 45, loss: 0.5373765826225281
In epoch 50, loss: 0.5344535708427429
In epoch 55, loss: 0.5323731899261475
In epoch 60, loss: 0.5307518839836121
In epoch 65, loss: 0.5293473601341248
In epoch 70, loss: 0.5279945135116577
In epoch 75, loss: 0.5265880227088928
In epoch 80, loss: 0.5250661969184875
In epoch 85, loss: 0.5234071612358093
In epoch 90, loss: 0.5216544270515442
In epoch 95, loss: 0.51988285779953


In [ ]:
h.shape

torch.Size([9993, 16])

In [ ]:
# 벡터 유사도 측정 함수
def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

In [ ]:
cos_sim(h[9743].detach().numpy(),h[9742].detach().numpy() )

0.9466203

In [ ]:
cos_sim(h[9743].detach().numpy(),h[9744].detach().numpy() )

0.5256134

In [ ]:
cos_sim(h[9743].detach().numpy(),h[7648].detach().numpy() )

0.60286206

In [ ]:
cos_sim(h[9743].detach().numpy(),h[7761].detach().numpy() )

0.88143265

In [ ]:
cos_sim(h[9743].detach().numpy(),h[6151].detach().numpy() )

0.8899083

In [ ]:
cos_sim(h[9743].detach().numpy(),h[6255].detach().numpy() )

0.1227138

In [ ]:
cos_sim(h[2].detach().numpy(), h[7890].detach().numpy() )

-0.58610535